# Collate, Clean data and publish as SQLite Database

## Notes:

1. Police data presented at econclass level, because the departments does not use main government accounting system (BAS). Integration has a 6 weeks lag.
2. Defence data presented at econclass level, because the department does not use main government accounting system (BAS). Integration has a 6 weeks lag and only at the item level.
3. Parliament excluded from dataset because the department does not use the main government accounting system (BAS).
4. SCoA report tables separate Direct Charges from Appropriated Fund spending, and the same approach has been used here by filtering on FUND_LEVEL_2
5. Debt Service Costs and National Revenue Fund Payments (under National Treasury) are included at econclass level since these payments are not made through the main government accounting system (BAS).

## TODO:

 - [ ] Download files directly from government website in notebook
 - [ ] Tests, both in ETL process as well as main repo
 - [ ] Implement logging

<p style="color:rgb(255,0,0);">Discrepancies:</p>

<ol>
    <li>2020-21-Q3 National.csv link doesn't direct to right direction</li>
</ol>

## Imports

In [ ]:
import os
import pathlib
import sqlite3
import urllib
import warnings
from zipfile import ZipFile
from urllib.parse import urljoin

import requests
import numpy as np
import pandas as pd
from rarfile import RarFile
from tqdm import tqdm
from io import BytesIO

warnings.simplefilter(action="ignore", category=FutureWarning)
pd.set_option("display.max_columns", None)

## Utility Functions

In [ ]:
def read_dirty_treasury_csv(
    path: pathlib.PurePath, cols_to_use: list, dtypes: dict
) -> pd.DataFrame:

    df = pd.read_csv(
        path,
        encoding="Windows-1252",
        sep=",",
        usecols=cols_to_use,
        skip_blank_lines=True,
        error_bad_lines=False,
        warn_bad_lines=False,  # Need to deal with these eventually
        low_memory=False,
        dtype=dtypes,
        thousands=" ",
        na_values="NaN",
        skipinitialspace=True,
    )

    return df

In [ ]:
TREASURY_BASE_URL = "http://www.treasury.gov.za/statistics/Quarterly%20spending%20data/"
DATA_PATH = pathlib.Path("/home/jovyan/work/data")
FILE_ENDPOINTS = {
    "2019_Q1": "2019/SCOA_for_web.rar",
    "2019_Q2": "2019/2019-20-Q2%20National.rar",
    "2019_Q3": "2019/2019-20-Q3%20National.rar",
    "2019_Q4": "2019/2019-20-Q4%20National.rar",
    "2020_Q1": "2020/2020-21-Q1%20National.rar",
    "2020_Q2": "2020/2020-21-Q2%20National.zip",
    # "2020_Q3":"2020/2020-21-Q3%20National.zip", # This link is broken on website
    "2020_Q4": "2020/2020-21-Q4%20National.zip",
    "2021_Q1": "2021/2021-22-Q1%20National.zip",
}
FILE_ENDPOINTS_FULL = {
    k: urljoin(TREASURY_BASE_URL, v) for k, v in FILE_ENDPOINTS.items()
}

# function to download file from treasury's website
def download_file(url: str, path_to_save: pathlib.PurePath) -> None:
    """
    Adapted from
    https://stackoverflow.com/questions/6861323/download-and-unzip-file-with-python
    """

    suffix = pathlib.Path(url).suffix
    if suffix == ".rar":

        print(f"Downloading {url}...", end="")

        response = requests.get(url).content

        print("DONE!")

        print(f"Parsing downloaded {pathlib.Path(url).name}")

        bytes_in_memory = BytesIO(response)
        rar_file_object = RarFile(bytes_in_memory, "r")
        
        # open csv file
        csv_file = rar_file_object.open(csv_file_name := rar_file_object.namelist()[0])
        print(csv_file_name)
        csv_content_bytes = csv_file.read()

        print(f"{csv_file_name} finished downloading. Saving as {str(path_to_save)}")

        with open(path_to_save, "wb") as fh:
            fh.write(csv_content_bytes)
            
        # cleanup
        bytes_in_memory.close()
        rar_file_object.close()

    elif suffix == ".zip":

        print(f"Downloading {url}...", end="")

        filehandle, _ = urllib.request.urlretrieve(url)

        print("DONE!")

        print(f"Parsing downloaded {url}")

        zip_file_object = zipfile.ZipFile(filehandle, "r")
        csv_file_name = zip_file_object.namelist()[0]
        csv_file = zip_file_object.open(csv_file_name)
        csv_content = csv_file.read()

        print(f"{csv_file_name} finished downloading. Saving as {str(path_to_save)}")

        with open(path_to_save, "wb") as fh:
            fh.write(csv_content)
            
    else:
        print(f"Couldn't download and parse {url}!")

In [ ]:
csvs_to_save = [
    DATA_PATH / "{}.csv".format(f) for f in sorted(FILE_ENDPOINTS_FULL.keys())
]

for csv_to_save in csvs_to_save:
    timeperiod = csv_to_save.name.strip(".csv")

    download_file(FILE_ENDPOINTS_FULL[timeperiod], csv_to_save)

In [ ]:
pathlib.Path(
    "http://www.treasury.gov.za/statistics/Quarterly%20spending%20data/2019/SCOA_for_web.rar"
).suffix

In [ ]:
metadata_dict = {
    "Description": {
        "Vote No#": "Vote number of the department",
        "Department": "Department name",
        "Programme No#": "Programme number as specified in the ENE",
        "Programme": "Programme name within the department",
        "Subprogramme No#": "Subprogramme number as specified in the ENE",
        "Subprogramme": "subprogramme name within the parent programme",
        "econClass_L1": "economic classification of spending level 1",
        "econClass_L2": "economic classification of spending level 2",
        "econClass_L3": "economic classification of spending level 3",
        "econClass_L4": "economic classification of spending level 4",
        "econClass_L5": "economic classification of spending level 5",
        "IYM_econLowestLevel": "lowest level economic classification of spending as submitted in the IYM to the Treasury",
        "Item_Lowest_Level": "SCoA Item segment lowest level",
        "Assets_Level_1": "SCoA Asset segment level 1",
        "Assets_Level_2": "SCoA Asset segment level 2",
        "Assets_Level_3": "SCoA Asset segment level 3",
        "Assets_Level_4": "SCoA Asset segment level 4",
        "Assets_Level_5": "SCoA Asset segment level 5",
        "Assets_Level_6": "SCoA Asset segment level 6",
        "Assets_Lowest_Level": "SCoA Asset segment lowest level",
        "Project_Level_1": "SCoA Project segment level 1",
        "Project_Level_2": "SCoA Project segment level 2",
        "Project_Level_3": "SCoA Project segment level 3",
        "Project_Level_4": "SCoA Project segment level 4",
        "Project_Level_5": "SCoA Project segment level 5",
        "Project_Level_6": "SCoA Project segment level 6",
        "Project_Level_7": "SCoA Project segment level 7",
        "Project_Level_8": "SCoA Project segment level 8",
        "Project_Level_9": "SCoA Project segment level 9",
        "Project_Level_10": "SCoA Project segment level 10",
        "Project_Level_11": "SCoA Project segment level 11",
        "Project_Lowest_Level": "SCoA Project segment lowest level",
        "Responsibility_Level_2": "SCoA Responsibility segment level 2",
        "Responsibility_Level_3": "SCoA Responsibility segment level 3",
        "Responsibility_Level_4": "SCoA Responsibility segment level 4",
        "Responsibility_Level_5": "SCoA Responsibility segment level 5",
        "Responsibility_Level_6": "SCoA Responsibility segment level 6",
        "Responsibility_Level_7": "SCoA Responsibility segment level 7",
        "Responsibility_Level_8": "SCoA Responsibility segment level 8",
        "Responsibility_Level_9": "SCoA Responsibility segment level 9",
        "Responsibility_Level_10": "SCoA Responsibility segment level 10",
        "Responsibility_Level_11": "SCoA Responsibility segment level 11",
        "Responsibility_Level_12": "SCoA Responsibility segment level 12",
        "Responsibility_Level_13": "SCoA Responsibility segment level 13",
        "Responsibility_Level_14": "SCoA Responsibility segment level 14",
        "Responsibility_Level_15": "SCoA Responsibility segment level 15",
        "Responsibility_Lowest_Level": "SCoA Responsibility segment lowest level",
        "Fund_Level_1": "SCoA Fund segment level 1",
        "Fund_Level_2": "SCoA Fund segment level 2",
        "Fund_Level_3": "SCoA Fund segment level 3",
        "Fund_Level_4": "SCoA Fund segment level 4",
        "Fund_Level_5": "SCoA Fund segment level 5",
        "Fund_Level_6": "SCoA Fund segment level 6",
        "Fund_Level_7": "SCoA Fund segment level 7",
        "Fund_Level_8": "SCoA Fund segment level 8",
        "Fund_Lowest_Level": "SCoA Fund segment lowest level",
        "Infrastructure_Level_1": "SCoA Infrastructure segment level 1",
        "Infrastructure_Level_2": "SCoA Infrastructure segment level 2",
        "Infrastructure_Level_3": "SCoA Infrastructure segment level 3",
        "Infrastructure_Level_4": "SCoA Infrastructure segment level 4",
        "Infrastructure_Level_5": "SCoA Infrastructure segment level 5",
        "Infrastructure_Level_6": "SCoA Infrastructure segment level 6",
        "Infrastructure_Lowest_Level": "SCoA Infrastructure segment lowest level",
        "Item_Level_1": "SCoA Item segment level 1",
        "Item_Level_2": "SCoA Item segment level 2",
        "Item_Level_3": "SCoA Item segment level 3",
        "Item_Level_4": "SCoA Item segment level 4",
        "Item_Level_5": "SCoA Item segment level 5",
        "Item_Level_6": "SCoA Item segment level 6",
        "Item_Level_7": "SCoA Item segment level 7",
        "Item_Level_8": "SCoA Item segment level 8",
        "Regional_ID_Level_1": "SCoA Regional segment level 1",
        "Regional_ID_Level_2": "SCoA Regional segment level 2",
        "Regional_ID_Level_3": "SCoA Regional segment level 3",
        "Regional_ID_Level_4": "SCoA Regional segment level 4",
        "Regional_ID_Level_5": "SCoA Regional segment level 5",
        "Regional_ID_Level_6": "SCoA Regional segment level 6",
        "Regional_ID_Level_7": "SCoA Regional segment level 7",
        "Regional_ID_Level_8": "SCoA Regional segment level 8",
        "Regional_ID_Lowest_Level": "SCoA Regional segment lowest level",
        "Budget": "Budgeted amount",
        "AdjustmentBudget": "Adjustments to the original budget",
        "April": "April expenditure",
        "May": "May expenditure",
        "June": "June expenditure",
        "July": "July expenditure",
        "August": "August expenditure",
        "September": "September expenditure",
        "October": "October expenditure",
        "November": "November expenditure",
        "December": "December expenditure",
        "January": "January expenditure",
        "February": "February expenditure",
        "March": "March expenditure",
        "Q1": "Quarter one expenditure (April + May + June)",
        "Q2": "Quarter two expenditure (July + August + Spetember)",
        "Q3": "Quarter one expenditure (October + November + December)",
        "Q4": "Quarter one expenditure (January + February + March)",
        "Financial_Year": "Financial year ( period April - March)",
    },
    "Data type": {
        "Vote No#": "number",
        "Department": "text",
        "Programme No#": "number",
        "Programme": "text",
        "Subprogramme No#": "text",
        "Subprogramme": "text",
        "econClass_L1": "text",
        "econClass_L2": "text",
        "econClass_L3": "text",
        "econClass_L4": "text",
        "econClass_L5": "text",
        "IYM_econLowestLevel": "text",
        "Item_Lowest_Level": "text",
        "Assets_Level_1": "text",
        "Assets_Level_2": "text",
        "Assets_Level_3": "text",
        "Assets_Level_4": "text",
        "Assets_Level_5": "text",
        "Assets_Level_6": "text",
        "Assets_Lowest_Level": "text",
        "Project_Level_1": "text",
        "Project_Level_2": "text",
        "Project_Level_3": "text",
        "Project_Level_4": "text",
        "Project_Level_5": "text",
        "Project_Level_6": "text",
        "Project_Level_7": "text",
        "Project_Level_8": "text",
        "Project_Level_9": "text",
        "Project_Level_10": "text",
        "Project_Level_11": "text",
        "Project_Lowest_Level": "text",
        "Responsibility_Level_2": "text",
        "Responsibility_Level_3": "text",
        "Responsibility_Level_4": "text",
        "Responsibility_Level_5": "text",
        "Responsibility_Level_6": "text",
        "Responsibility_Level_7": "text",
        "Responsibility_Level_8": "text",
        "Responsibility_Level_9": "text",
        "Responsibility_Level_10": "text",
        "Responsibility_Level_11": "text",
        "Responsibility_Level_12": "text",
        "Responsibility_Level_13": "text",
        "Responsibility_Level_14": "text",
        "Responsibility_Level_15": "text",
        "Responsibility_Lowest_Level": "text",
        "Fund_Level_1": "text",
        "Fund_Level_2": "text",
        "Fund_Level_3": "text",
        "Fund_Level_4": "text",
        "Fund_Level_5": "text",
        "Fund_Level_6": "text",
        "Fund_Level_7": "text",
        "Fund_Level_8": "text",
        "Fund_Lowest_Level": "text",
        "Infrastructure_Level_1": "text",
        "Infrastructure_Level_2": "text",
        "Infrastructure_Level_3": "text",
        "Infrastructure_Level_4": "text",
        "Infrastructure_Level_5": "text",
        "Infrastructure_Level_6": "text",
        "Infrastructure_Lowest_Level": "text",
        "Item_Level_1": "text",
        "Item_Level_2": "text",
        "Item_Level_3": "text",
        "Item_Level_4": "text",
        "Item_Level_5": "text",
        "Item_Level_6": "text",
        "Item_Level_7": "text",
        "Item_Level_8": "text",
        "Regional_ID_Level_1": "text",
        "Regional_ID_Level_2": "text",
        "Regional_ID_Level_3": "text",
        "Regional_ID_Level_4": "text",
        "Regional_ID_Level_5": "text",
        "Regional_ID_Level_6": "text",
        "Regional_ID_Level_7": "text",
        "Regional_ID_Level_8": "text",
        "Regional_ID_Lowest_Level": "text",
        "Budget": "number",
        "AdjustmentBudget": "number",
        "April": "number",
        "May": "number",
        "June": "number",
        "July": "number",
        "August": "number",
        "September": "number",
        "October": "number",
        "November": "number",
        "December": "number",
        "January": "number",
        "February": "number",
        "March": "number",
        "Q1": "number",
        "Q2": "number",
        "Q3": "number",
        "Q4": "number",
        "Financial_Year": "text",
    },
    "Sample attributes": {
        "Vote No#": "1 - 40",
        "Department": "The Presidency, Home Affairs",
        "Programme No#": "1 - 10",
        "Programme": "Administration, Citizen Affairs",
        "Subprogramme No#": "1 - 15",
        "Subprogramme": "Ministry, Service delivery to Provinces",
        "econClass_L1": "Payments, Receipts",
        "econClass_L2": "Current payments, Payments for capital assets",
        "econClass_L3": "Goods and services, Buildings and other structures",
        "econClass_L4": "Consultants: Business and advisory services, Inventory: Fuel, oil and gas",
        "econClass_L5": "Subsidies on products and production (pe)",
        "IYM_econLowestLevel": "Administrative fees, Advertising",
        "Item_Lowest_Level": "A&S/O/S:LEGAL SERVICE, ",
        "Assets_Level_1": np.nan,
        "Assets_Level_2": np.nan,
        "Assets_Level_3": np.nan,
        "Assets_Level_4": np.nan,
        "Assets_Level_5": np.nan,
        "Assets_Level_6": np.nan,
        "Assets_Lowest_Level": np.nan,
        "Project_Level_1": np.nan,
        "Project_Level_2": np.nan,
        "Project_Level_3": np.nan,
        "Project_Level_4": np.nan,
        "Project_Level_5": np.nan,
        "Project_Level_6": np.nan,
        "Project_Level_7": np.nan,
        "Project_Level_8": np.nan,
        "Project_Level_9": np.nan,
        "Project_Level_10": np.nan,
        "Project_Level_11": np.nan,
        "Project_Lowest_Level": np.nan,
        "Responsibility_Level_2": np.nan,
        "Responsibility_Level_3": np.nan,
        "Responsibility_Level_4": np.nan,
        "Responsibility_Level_5": np.nan,
        "Responsibility_Level_6": np.nan,
        "Responsibility_Level_7": np.nan,
        "Responsibility_Level_8": np.nan,
        "Responsibility_Level_9": np.nan,
        "Responsibility_Level_10": np.nan,
        "Responsibility_Level_11": np.nan,
        "Responsibility_Level_12": np.nan,
        "Responsibility_Level_13": np.nan,
        "Responsibility_Level_14": np.nan,
        "Responsibility_Level_15": np.nan,
        "Responsibility_Lowest_Level": np.nan,
        "Fund_Level_1": "EXPENDITURE:VOTED",
        "Fund_Level_2": np.nan,
        "Fund_Level_3": np.nan,
        "Fund_Level_4": np.nan,
        "Fund_Level_5": np.nan,
        "Fund_Level_6": np.nan,
        "Fund_Level_7": np.nan,
        "Fund_Level_8": np.nan,
        "Fund_Lowest_Level": np.nan,
        "Infrastructure_Level_1": np.nan,
        "Infrastructure_Level_2": np.nan,
        "Infrastructure_Level_3": np.nan,
        "Infrastructure_Level_4": np.nan,
        "Infrastructure_Level_5": np.nan,
        "Infrastructure_Level_6": np.nan,
        "Infrastructure_Lowest_Level": np.nan,
        "Item_Level_1": np.nan,
        "Item_Level_2": np.nan,
        "Item_Level_3": np.nan,
        "Item_Level_4": np.nan,
        "Item_Level_5": np.nan,
        "Item_Level_6": np.nan,
        "Item_Level_7": np.nan,
        "Item_Level_8": np.nan,
        "Regional_ID_Level_1": np.nan,
        "Regional_ID_Level_2": np.nan,
        "Regional_ID_Level_3": np.nan,
        "Regional_ID_Level_4": np.nan,
        "Regional_ID_Level_5": np.nan,
        "Regional_ID_Level_6": np.nan,
        "Regional_ID_Level_7": np.nan,
        "Regional_ID_Level_8": np.nan,
        "Regional_ID_Lowest_Level": np.nan,
        "Budget": "0, 5000",
        "AdjustmentBudget": "50, -200",
        "April": "1428, 7, -30",
        "May": "1428, 7, -31",
        "June": "1428, 7, -32",
        "July": "1428, 7, -33",
        "August": "1428, 7, -34",
        "September": "1428, 7, -35",
        "October": "1428, 7, -36",
        "November": "1428, 7, -37",
        "December": "1428, 7, -38",
        "January": "1428, 7, -39",
        "February": "1428, 7, -40",
        "March": "1428, 7, -41",
        "Q1": "20937, 55",
        "Q2": "20937, 56",
        "Q3": "20937, 57",
        "Q4": "20937, 58",
        "Financial_Year": "2019/20",
    },
}

In [ ]:
metadata = pd.DataFrame(metadata_dict)

intended_map_types = (
    metadata["Data type"]
    .map(
        {
            "number": np.float32,
            "text": "object",
        }
    )
    .to_dict()
)

# data is super dirty. so fix_map_types
map_types_temp = {k: "object" for k, v in intended_map_types.items()}

# drop empty columns
to_drop = metadata[metadata["Sample attributes"].isna()].index.to_list()

columns_to_keep = set(map_types_temp.keys()) - set(to_drop)
csvs_to_save = [
    DATA_PATH / "{}.csv".format(f) for f in sorted(FILE_ENDPOINTS_FULL.keys())
]

In [ ]:
csvs_to_read

In [ ]:
df = pd.concat(
    [
        read_dirty_treasury_csv(filepath, columns_to_keep, map_types_temp)
        for filepath in tqdm(csvs_to_read)
    ],
    axis=0,
    ignore_index=True,
)

original_col_order = list(df.columns)

In [ ]:
%%time

# convert num cols to nums, coercing errors
intended_num_cols = {k for k, v in intended_map_types.items() if v == np.float32}

# remove spaces from num cols
numeric_cols = df.loc[:, intended_num_cols].apply(
    lambda col: col.str.strip(" "), axis=0
)

# coerce to numeric
numeric_cols = numeric_cols.apply(
    lambda col: pd.to_numeric(col, errors="coerce", downcast="float"), axis=0
)

# bring back to main dataframe
df = df.drop(columns=intended_num_cols)
df = pd.concat([df, numeric_cols], axis=1)

In [ ]:
# reorder columns to original order
df = df.loc[:, original_col_order]

In [ ]:
%%time

# https://github.com/deepak7376/downcast/blob/master/src/downcast.py
def reduce(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i, t in enumerate(types):
        if "int" in str(t):
            if (
                df[cols[i]].min() > np.iinfo(np.int8).min
                and df[cols[i]].max() < np.iinfo(np.int8).max
            ):
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif (
                df[cols[i]].min() > np.iinfo(np.int16).min
                and df[cols[i]].max() < np.iinfo(np.int16).max
            ):
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif (
                df[cols[i]].min() > np.iinfo(np.int32).min
                and df[cols[i]].max() < np.iinfo(np.int32).max
            ):
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif "float" in str(t):
            if (
                df[cols[i]].min() > np.finfo(np.float16).min
                and df[cols[i]].max() < np.finfo(np.float16).max
            ):
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif (
                df[cols[i]].min() > np.finfo(np.float32).min
                and df[cols[i]].max() < np.finfo(np.float32).max
            ):
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == np.object:
            if cols[i] == "date":
                df[cols[i]] = pd.to_datetime(df[cols[i]], format="%Y-%m-%d")
            else:
                df[cols[i]] = df[cols[i]].astype("category")
    return df


df = reduce(df)

In [ ]:
df.info(memory_usage="deep")

In [ ]:
missings = df.isna().mean(axis=0).sort_values()
missings

In [ ]:
metadata.T

## Export to SQLite Database

In [ ]:
DB_PATH = "/home/jovyan/work/data/processed/SAGovBudget.sqlite"

try:
    os.remove(DB_PATH)
except FileNotFoundError:
    pass

In [ ]:
%%time

conn = sqlite3.connect(DB_PATH)

# write dataframe to sqlitedb
df.to_sql(
    name="National", con=conn, if_exists="replace", index=False, chunksize=100_000
)
metadata.to_sql(name="Metadata", con=conn, if_exists="replace", index=False)

conn.close()